In [1]:
import warnings
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearnex import patch_sklearn, config_context
patch_sklearn()
warnings.filterwarnings('ignore')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import os
ipynb_path = os.getcwd()
src_path = os.path.join(ipynb_path, 'src/')
input_path = os.path.join(ipynb_path,"input/")

In [3]:

import warnings
import os
import sys

import scipy.stats as spst

sys.path.append(src_path)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask
import dask.dataframe as dd
from windpowerlib.wind_speed import logarithmic_profile
from src.utils import uv_to_wsd # 윈도우에서는 앞에 src를 뺄것

In [4]:
power_2020 = pd.read_parquet(input_path + "dynamic_report_ewp02_2020_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power_2021 = pd.read_parquet(input_path + "dynamic_report_ewp02_2021_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power_2022 = pd.read_parquet(input_path + "dynamic_report_ewp02_2022_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power = pd.concat([power_2020, power_2021, power_2022], ignore_index=True)

gj_y = pd.read_parquet(input_path + "train_y.parquet").rename({'end_datetime': 'dt'}, axis=1)
ldaps = pd.read_parquet(input_path + "train_ldaps_gyeongju.parquet")

print("Power: ", power.shape)
print("train_y: ", gj_y.shape)
print("LDAPS: ", ldaps.shape)

Power:  (155528, 29)
train_y:  (52608, 4)
LDAPS:  (235818, 15)


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit

# yongmin's functions
from src.utils import DataConnector
from src.metric import NMAE
from src.data_processor import *

# model import
import xgboost as xgb
from xgboost import XGBRegressor


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [6]:
# 파이프라인 구성 및 적용
DataPipeline = Pipeline([
    ('uv_transform', UVTransformer('wind_u_10m', 'wind_v_10m')),
    ('wind_transform', WindTransformer('wind_speed', 10, 100, ldaps['surf_rough'].mean())),
    ('feature_engineering', FeatureTransformer()),
])

# 파이프라인을 이용하여 ldaps 데이터 변환
ldaps_transformed = DataPipeline.fit_transform(ldaps)

print(ldaps_transformed.shape)


(235818, 23)


In [7]:
average_ldaps = ldaps_transformed.drop('turbine_id', axis=1).groupby('dt').mean()
average_ldaps.columns = average_ldaps.columns.str.replace(r'[<>\[\]]', '_', regex=True)
average_ldaps.columns = average_ldaps.columns.str.replace(r'[^\w]', '_', regex=True)
average_ldaps.columns = average_ldaps.columns.str.replace(r'__+', '_', regex=True)

In [8]:
average_ldaps.reset_index(inplace=True)


In [9]:
average_ldaps['dt'] = pd.to_datetime(average_ldaps['dt']).dt.tz_localize(None)
gj_y['dt'] = pd.to_datetime(gj_y['dt']).dt.tz_localize(None)
avg_data = pd.merge(average_ldaps, gj_y, on='dt', how='inner')

In [10]:
avg_data_sorted = avg_data.sort_values(['dt', 'plant_name', 'energy_kwh'], ascending=[True, True, False])
avg_data_cleaned = avg_data_sorted.drop_duplicates(subset=['dt', 'plant_name'], keep='first')

In [11]:
avg_data_cleaned = avg_data.drop_duplicates(subset=['dt'], keep='first')
avg_data = avg_data_cleaned

In [12]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


def get_trasforms_datas(merged_data, numeric_columns, target):
    z_scaler = StandardScaler()
    minmax_scaler = MinMaxScaler()
    
    x_train = merged_data.loc[merged_data['dt'].between('2020-01-01', '2020-12-31', inclusive='left'), numeric_columns]
    x_test = merged_data.loc[merged_data['dt'].between('2021-01-01', '2022-12-31', inclusive='left'), numeric_columns]

    y_train = merged_data.loc[merged_data['dt'].between('2020-01-01', '2020-12-31', inclusive='left'), target].shift(periods = -24)
    y_test = merged_data.loc[merged_data['dt'].between('2021-01-01', '2022-12-31', inclusive='left'), target].shift(periods = -24)
    #y_train = y_train.dropna()
    #y_test = y_test.dropna()

    x_train = x_train.iloc[:-24]
    y_train = y_train.iloc[:-24]

    x_test = x_test.iloc[:-24]
    y_test = y_test.iloc[:-24]

    # Min-Max Scaling
    x_train_m = minmax_scaler.fit_transform(x_train)
    x_train_m = pd.DataFrame(x_train_m, columns=x_train.columns)
    x_test_m = minmax_scaler.transform(x_test)
    x_test_m = pd.DataFrame(x_test_m, columns=x_train.columns)

    # Standard Scaling
    x_train_z = z_scaler.fit_transform(x_train)
    x_train_z = pd.DataFrame(x_train_z, columns=x_train.columns)
    x_test_z = z_scaler.transform(x_test)
    x_test_z = pd.DataFrame(x_test_z, columns=x_train.columns)

    return x_train, x_test, x_train_m, x_test_m, x_train_z, x_test_z, y_train, y_test

In [13]:
from sklearn.cluster import KMeans

# 이제 특징 생성에 클러스터링 결과는 보지 않을 예정.
def addKmeansFeature(train_data, test_data):
    pd.options.mode.chained_assignment = None

    for n_clusters in range(2, 7):  # 2부터 6까지 클러스터 생성
        kmeans = KMeans(n_clusters=n_clusters, n_init=10)

        train_data[f'cluster_{n_clusters}'] = kmeans.fit_predict(train_data[['wind_speed', 'wind_direction']])
        
        test_data[f'cluster_{n_clusters}'] = kmeans.predict(test_data[['wind_speed', 'wind_direction']])

    return train_data, test_data
from sklearn.decomposition import PCA

def addPCAFeature(train_data, test_data):
    # PCA 적용할 특징 열 선택 (u, v 성분)
    wind_features = ['storm_u_5m', 'storm_v_5m', 'wind_u_10m', 'wind_v_10m', 
                     'wind_speed', 'wind_direction']
    
    # 훈련 데이터에서 PCA 학습
    pca = PCA(n_components=2)
    pca_train = pca.fit_transform(train_data[wind_features])
    
    # 훈련 데이터에 주성분 추가
    train_data['PC1'] = pca_train[:, 0]
    train_data['PC2'] = pca_train[:, 1]
    
    # 테스트 데이터에 PCA 적용
    pca_test = pca.transform(test_data[wind_features])
    test_data['PC1'] = pca_test[:, 0]
    test_data['PC2'] = pca_test[:, 1]

    # PCA 설명력 확인
    explained_variance = pca.explained_variance_ratio_
    print(f"PC1 설명력: {explained_variance[0]}")
    print(f"PC2 설명력: {explained_variance[1]}")

    return train_data, test_data

from sklearn_extra.cluster import KMedoids

def addKMedoidsFeature(train_data, test_data):
    pd.options.mode.chained_assignment = None

    for n_clusters in range(2, 7):  # 2부터 6까지 클러스터 생성
        kmedoids = KMedoids(n_clusters=n_clusters, random_state=42)

        # 훈련 데이터에 K-Medoids 클러스터링 적용
        train_data[f'medoid_cluster_{n_clusters}'] = kmedoids.fit_predict(train_data[['wind_speed', 'wind_direction']])

        # 테스트 데이터에 학습된 K-Medoids 모델 적용
        test_data[f'medoid_cluster_{n_clusters}'] = kmedoids.predict(test_data[['wind_speed', 'wind_direction']])

    return train_data, test_data


In [14]:
numeric_columns = avg_data.select_dtypes(include=['number']).columns.tolist()

In [15]:
x_train, x_test, x_train_m, x_test_m, x_train_z, x_test_z, y_train, y_test = get_trasforms_datas(avg_data, numeric_columns, 'energy_kwh')

In [16]:
x_train, x_test = addKmeansFeature(x_train, x_test)
x_train_m, x_test_m = addKmeansFeature(x_train_m, x_test_m)
x_train_z, x_test_z = addKmeansFeature(x_train_z, x_test_z)
print('kmean 적용 완료')
x_train, x_test = addPCAFeature(x_train, x_test)
x_train_m, x_test_m = addPCAFeature(x_train_m, x_test_m)
x_train_z, x_test_z = addPCAFeature(x_train_z, x_test_z)
print('pca 적용 완료')

x_train, x_test = addKMedoidsFeature(x_train, x_test)
x_train_m, x_test_m = addKMedoidsFeature(x_train_m, x_test_m)
x_train_z, x_test_z = addKMedoidsFeature(x_train_z, x_test_z)
print('kmedoid 적용 완료')


kmean 적용 완료
PC1 설명력: 0.9982813596725464
PC2 설명력: 0.0008244864293374121
PC1 설명력: 0.7519216586247816
PC2 설명력: 0.1166772803574429
PC1 설명력: 0.3335494150013585
PC2 설명력: 0.3287007000824763
pca 적용 완료
kmedoid 적용 완료


In [17]:
x_dict = {
    # 원본 데이터만 사용
    'original': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                 'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                 'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m'],

    # PCA 추가
    'pca_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                 'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                 'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'PC1', 'PC2'],

    # 클러스터(2~6) 추가
    'cluster_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                     'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                     'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                     'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6'],

    # PCA + 클러스터(2~6) 추가
    'pca_and_cluster': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                        'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                        'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                        'PC1', 'PC2', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6'],

    # 클러스터 개수에 따른 경우
    'cluster_2_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_2'],
    
    'cluster_3_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_3'],
    
    'cluster_4_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_4'],
    
    'cluster_5_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_5'],
    
    'cluster_6_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_6'],

    # KMedoids 추가
    'kmedoids_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                      'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                      'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                      'medoid_cluster_2', 'medoid_cluster_3', 'medoid_cluster_4', 'medoid_cluster_5', 'medoid_cluster_6'],

    # PCA + KMedoids 추가
    'pca_and_kmedoids': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                         'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                         'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                         'PC1', 'PC2', 'medoid_cluster_2', 'medoid_cluster_3', 'medoid_cluster_4', 'medoid_cluster_5', 'medoid_cluster_6'],

    # KMedoids 클러스터 개수에 따른 경우
    'medoid_cluster_2_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_2'],

    'medoid_cluster_3_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_3'],

    'medoid_cluster_4_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_4'],

    'medoid_cluster_5_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_5'],

    'medoid_cluster_6_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_6']

}


In [18]:
import torch

In [19]:
from src.deepTrain_roughVer.Analysis_WindTurbine.Model.RNNs import RNN,LSTM,GRU

In [20]:
from torch.utils.data import DataLoader, TensorDataset
from torch import nn, optim

In [24]:
from sklearn.metrics import r2_score, mean_absolute_error

def NMAE(y_true, y_pred):
    """NMAE 계산 함수."""
    return mean_absolute_error(y_true, y_pred) / (sum(abs(y_true)) / len(y_true)) * 100

def train_model(save_dir, model, x_train, y_train, x_test, y_test, epochs=200, batch_size=30, lr=0.001):
    if not torch.cuda.is_available():
        raise RuntimeError("cuda is not available. Exiting...")

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    model = model.cuda()

    x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32).cuda()
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1).cuda()
    x_test_tensor = torch.tensor(x_test.values, dtype=torch.float32).cuda()
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1).cuda()

    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    best_val_loss = float('inf')
    best_y_pred_list = []
    best_y_true_list = []
    train_losses = []
    val_losses = []

    csv_data = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.cuda(), targets.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        y_pred_list = []
        y_true_list = []
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.cuda(), targets.cuda()
                outputs = model(inputs)
                val_loss += criterion(outputs, targets).item()
                y_pred_list.extend(outputs.cpu().numpy())
                y_true_list.extend(targets.cpu().numpy())

        y_pred_list = np.array(y_pred_list).flatten()
        y_true_list = np.array(y_true_list).flatten()

        mae = mean_absolute_error(y_true_list, y_pred_list)
        nmae = NMAE(y_true_list, y_pred_list)
        r2 = r2_score(y_true_list, y_pred_list)

        train_losses.append(train_loss / len(train_loader))
        val_losses.append(val_loss / len(test_loader))

        csv_data.append([epoch + 1, train_loss / len(train_loader), val_loss / len(test_loader), mae, nmae, r2])

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_y_pred_list = y_pred_list
            best_y_true_list = y_true_list
            torch.save(model.state_dict(), os.path.join(save_dir, f"{model.__class__.__name__}_best_model.pth"))

        if (epoch + 1) % 10 == 0:
            torch.save(model.state_dict(), os.path.join(save_dir, f"{model.__class__.__name__}_epoch_{epoch+1}.pth"))

        print(f"Epoch [{epoch+1}/{epochs}]")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"Val Loss: {val_loss/len(test_loader):.4f}, MAE: {mae:.4f}, NMAE: {nmae:.4f}, R^2: {r2:.4f}")

    torch.save(model.state_dict(), os.path.join(save_dir, f"{model.__class__.__name__}_final_model.pth"))

    df = pd.DataFrame(csv_data, columns=["Epoch", "Train Loss", "Val Loss", "MAE", "NMAE", "R2"])
    df.to_csv(os.path.join(save_dir, "training_log.csv"), index=False)

    plt.figure()
    plt.plot(range(epochs), train_losses, label="Train Loss")
    plt.plot(range(epochs), val_losses, label="Val Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Loss plot")
    plt.savefig(os.path.join(save_dir, "loss_plot.png"))
    plt.close()

    plt.figure()
    plt.plot(best_y_true_list, label="ground truth")
    plt.plot(best_y_pred_list, label="Pred")
    plt.xlabel("Samples")
    plt.ylabel("Values")
    plt.legend()
    plt.title("Pred and Truth Compare")
    plt.savefig(os.path.join(save_dir, "best_pred_plot.png"))
    plt.close()

    # Log final performance
    with open(os.path.join(save_dir, "training_log.txt"), "w") as f:
        f.write(f"Final Validation Loss: {best_val_loss:.4f}\n")
        f.write(f"MAE: {mae:.4f}, NMAE: {nmae:.4f}, R^2: {r2:.4f}\n")
        f.write(f"devide capacity => MAE/20700: {(mae/20700):.4f}, MAE/79600: {(mae/79600):.4f}")

    print(f"Final Model saved with best validation loss: {best_val_loss:.4f}")
    print(f"MAE: {mae:.4f}, NMAE: {nmae:.4f}, R^2: {r2:.4f}")

In [26]:
for key in x_dict.keys():
    # ====================
    # Min-Max 정규화 데이터
    # ====================
    x_train_m_selected = x_train_m[x_dict[key]]
    x_test_m_selected = x_test_m[x_dict[key]]

    # RNN 모델 학습
    rnn_model = RNN(input_dim=x_train_m_selected.shape[1], hidden_dim=128)
    save_dir = os.path.join(ipynb_path, f'notebooks/test_basicRecurrent/{key}_RNN_m/')
    train_model(save_dir, rnn_model, x_train_m_selected, y_train, x_test_m_selected, y_test,2000)

    # LSTM 모델 학습
    lstm_model = LSTM(input_dim=x_train_m_selected.shape[1], hidden_dim=128)
    save_dir = os.path.join(ipynb_path, f'notebooks/test_basicRecurrent/{key}_LSTM_m/')
    train_model(save_dir, lstm_model, x_train_m_selected, y_train, x_test_m_selected, y_test,2000)

    # GRU 모델 학습
    lstm_model = GRU(input_dim=x_train_m_selected.shape[1], hidden_dim=128)
    save_dir = os.path.join(ipynb_path, f'notebooks/test_basicRecurrent/{key}_GRU_m/')
    train_model(save_dir, gru_model, x_train_m_selected, y_train, x_test_m_selected, y_test,2000)

    # ====================
    # z-정규화 데이터
    # ====================
    x_train_z_selected = x_train_z[x_dict[key]]
    x_test_z_selected = x_test_z[x_dict[key]]

    # RNN 모델 학습
    rnn_model = RNN(input_dim=x_train_z_selected.shape[1], hidden_dim=128)
    save_dir = os.path.join(ipynb_path, f'notebooks/test_basicRecurrent/{key}_RNN_z/')
    train_model(save_dir, rnn_model, x_train_z_selected, y_train, x_test_z_selected, y_test,2000)

    # LSTM 모델 학습
    lstm_model = LSTM(input_dim=x_train_z_selected.shape[1], hidden_dim=128)
    save_dir = os.path.join(ipynb_path, f'notebooks/test_basicRecurrent/{key}_LSTM_z/')
    train_model(save_dir, lstm_model, x_train_z_selected, y_train, x_test_z_selected, y_test,2000)

    # GRU 모델 학습
    gru_model = GRU(input_dim=x_train_z_selected.shape[1], hidden_dim=128)
    save_dir = os.path.join(ipynb_path, f'notebooks/test_basicRecurrent/{key}_GRU_z/')
    train_model(save_dir, gru_model, x_train_z_selected, y_train, x_test_z_selected, y_test,2000)


Epoch [1/2000]
Train Loss: 85511134.2591
Val Loss: 79098009.1164, MAE: 6528.5283, NMAE: 99.5328, R^2: -1.1638
Epoch [2/2000]
Train Loss: 85036494.5830
Val Loss: 78673838.6599, MAE: 6502.4180, NMAE: 99.1347, R^2: -1.1522
Epoch [3/2000]
Train Loss: 84589209.4114
Val Loss: 78261422.0663, MAE: 6477.5381, NMAE: 98.7554, R^2: -1.1410
Epoch [4/2000]
Train Loss: 84152791.2271
Val Loss: 77856744.9660, MAE: 6453.5078, NMAE: 98.3890, R^2: -1.1299
Epoch [5/2000]
Train Loss: 83723354.5481
Val Loss: 77457558.1315, MAE: 6430.1758, NMAE: 98.0333, R^2: -1.1190
Epoch [6/2000]
Train Loss: 83299070.7011
Val Loss: 77062744.6563, MAE: 6407.4189, NMAE: 97.6864, R^2: -1.1082
Epoch [7/2000]
Train Loss: 82879007.2699
Val Loss: 76671678.2096, MAE: 6385.0996, NMAE: 97.3461, R^2: -1.0975
Epoch [8/2000]
Train Loss: 82462623.8441
Val Loss: 76283982.7792, MAE: 6363.2275, NMAE: 97.0126, R^2: -1.0869
Epoch [9/2000]
Train Loss: 82049578.7545
Val Loss: 75899395.9374, MAE: 6341.7729, NMAE: 96.6855, R^2: -1.0763
Epoch [10/

NameError: name 'gru_model' is not defined